Need to realize this:

conda install nomkl
pip install langchain==0.0.133

https://stackoverflow.com/questions/71024254/jupyter-kernel-dies-when-importing-pipeline-function-from-transformers-class-on
https://stackoverflow.com/questions/53014306/error-15-initializing-libiomp5-dylib-but-found-libiomp5-dylib-already-initial


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain import HuggingFaceHub
llm = HuggingFaceHub(repo_id = "google/flan-t5-xxl")

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 900,
    chunk_overlap  = 20,
    length_function = len
)

loader = PyPDFLoader("/Users/stephenhky/randomipynb/apologeticscorpus/Mere Christianity (C. S. Lewis).pdf")
pages = loader.load_and_split(text_splitter=text_splitter)

In [6]:
pages

[Document(page_content='MERE \nCHRISTIANITY \nC . S . LEWIS \nA revised and ampliﬁed edition,  \nwith a new introduction,  \nof the three books  \nBroadcast Talks , \nChristian Behaviour \nand \nBeyond Personality', metadata={'source': '/Users/stephenhky/randomipynb/apologeticscorpus/Mere Christianity (C. S. Lewis).pdf', 'page': 1}),
 Document(page_content='contents \npreface \nforeword \nbook 1. right and wrong as a clue to \nthe meaning of the universe \n1 the law of human nature 3 \n2 some objections 9 \n3 the reality of the law 16 \n4 what lies behind the law 21 \n5 we have cause to be uneasy 28 \nbook 2. what christians believe \n1 the rival conceptions of god 35 \n2 the invasion 40 \n3 the shocking alternative 47 \n4 the perfect penitent 53 \n5 the practical conclusion 60 \nbook 3. christian behaviour \n1 the three parts of morality 69 \n2 the ‘cardinal virtues’ 76 \n3 social morality 82 \n4 morality and psychoanalysis 88 \n5 sexual morality 94 \n6 christian marriage 104 v \nxv',

In [7]:
# pip install faiss-cpu
from langchain.vectorstores import FAISS

# create the vectorestore to use as the index
db = FAISS.from_documents(pages, embeddings)

In [18]:
from langchain.chains import RetrievalQA

retriever = db.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True)

query = "Is forgiveness easy?"
result = qa({"query": query})

print(result['result'])

It is going to be hard enough, anyway, but I think there are two things we can do


In [9]:
from langchain import ConversationChain

conversation = ConversationChain(llm=llm, verbose=True)

In [10]:

conversation.predict(input="Alice has a parrot.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Alice has a parrot.
AI:

> Finished chain.


'Is it a blue parrot?'

In [11]:

conversation.predict(input="Bob has two cats.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Alice has a parrot.
AI: Is it a blue parrot?
Human: Bob has two cats.
AI:

> Finished chain.


'Is one of them a black cat?'

In [12]:
conversation.predict(input="How many pets do Alice and Bob have?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Alice has a parrot.
AI: Is it a blue parrot?
Human: Bob has two cats.
AI: Is one of them a black cat?
Human: How many pets do Alice and Bob have?
AI:

> Finished chain.


'They have three pets.'

In [17]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True)


agent.run("When was Barack Obama born? How old was he in 2022?")



> Entering new AgentExecutor chain...


ValueError: Could not parse LLM output: `Barack Obama was born on August 4, 1961. He was 57 years old in 2022. Final`